In [ ]:
import pandas as pd

indexes = []
all_indexes = {}

all_indexes = pd.read_excel('../Data/Index_Data.xlsx', sheet_name=None)

for sheet_name, df in all_indexes.items():
    indexes.append(sheet_name)
    all_indexes[sheet_name] = df

df = all_indexes['BSESN']
df.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock_Splits,Percent_Change_In_Price,Momentum_Indicator,High_Low_Change,High_Open_Change,Low_Open_Change,Day,Month,Year
6803,2025-02-11 00:00:00+05:30,77384.976562,77387.281250,76030.593750,76293.601562,9800,0,0,-0.013170,-0.014103,0.017844,0.000030,-0.017502,11,2,2025
6804,2025-02-12 00:00:00+05:30,76188.242188,76459.718750,75388.390625,76171.078125,10400,0,0,-0.001606,-0.000225,0.014211,0.003563,-0.010498,12,2,2025
6805,2025-02-13 00:00:00+05:30,76201.101562,76764.531250,76013.429688,76138.968750,6900,0,0,-0.000422,-0.000815,0.009881,0.007394,-0.002463,13,2,2025
6806,2025-02-14 00:00:00+05:30,76388.992188,76483.062500,75439.640625,75939.210938,9300,0,0,-0.002624,-0.005888,0.013831,0.001231,-0.012428,14,2,2025
6807,2025-02-17 00:00:00+05:30,75641.406250,76041.960938,75294.757812,75996.859375,0,0,0,0.000759,0.004699,0.009924,0.005295,-0.004583,17,2,2025


In [ ]:
# !pip install pycaret

In [ ]:
import pycaret
import numpy as np
from sklearn.preprocessing import StandardScaler
from pycaret.regression import *

pycaret.__version__

'3.3.2'

In [ ]:
df['RSI'] = 100 - (100 / (1 + (df['Close'].diff().clip(lower=0).rolling(window=14).mean() / df['Close'].diff().clip(upper=0).abs().rolling(window=14).mean())))
df['Daily_Range'] = df['High'] - df['Low']
df['Close_Low_Ratio'] = (df['Close'] - df['Low']) / df['Low']
df['MACD'] = df['Close'].ewm(span=12, adjust=False).mean() - df['Close'].ewm(span=26, adjust=False).mean()
df['ATR'] = df['Daily_Range'].rolling(window=14).mean()
df['Rolling_Std_Dev'] = df['Close'].rolling(window=14).std()

In [ ]:
df = df[[
    'Close',
    'Day', 
    'Month', 
    'Year',
    'RSI',
    'Daily_Range',
    'Close_Low_Ratio',
    'MACD',
    'ATR',
    'Rolling_Std_Dev'
]]

# Initialize PyCaret setup
reg_setup = setup(data=df,
                  target='Close')

# Compare models to find the best one
best_model = compare_models()

print(best_model)

# Create the best model
model = create_model(best_model)

# Tune the model
tuned_model = tune_model(model)

# Evaluate the model
evaluate_model(tuned_model)

# Finalize the model
final_model = finalize_model(tuned_model)

# Save the model
save_model(final_model, 'final_model')

,Description,Value
0,Session id,818
1,Target,Close
2,Target type,Regression
3,Original data shape,"(6808, 19)"
4,Transformed data shape,"(6808, 19)"
5,Transformed train set shape,"(4765, 19)"
6,Transformed test set shape,"(2043, 19)"
7,Numeric features,18
8,Rows with missing values,0.2%
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,203.8979,107490.6461,327.1428,0.9997,0.0186,0.0120,1.7180
rf,Random Forest Regressor,301.9084,262700.1966,511.5908,0.9994,0.0333,0.0190,5.1010
xgboost,Extreme Gradient Boosting,388.1072,327235.4094,570.8006,0.9992,0.0400,0.0262,0.6440
lightgbm,Light Gradient Boosting Machine,421.0879,371560.1068,608.4755,0.9991,0.0441,0.0297,0.6460
dt,Decision Tree Regressor,350.2561,475131.1888,684.7813,0.9988,0.0449,0.0207,0.0870
gbr,Gradient Boosting Regressor,827.3914,1327352.2604,1151.6007,0.9968,0.0821,0.0567,2.0980
ada,AdaBoost Regressor,1731.3183,4511303.4457,2122.0402,0.9890,0.2363,0.1835,0.7510
lr,Linear Regression,4836.8455,44200554.8981,6645.1976,0.8925,0.6375,0.4215,0.5120
lasso,Lasso Regression,4857.2780,44714952.2823,6682.4352,0.8913,0.6325,0.4141,0.1050
llar,Lasso Least Angle Regression,4872.4330,45725565.8526,6752.9958,0.8889,0.6576,0.4092,0.0210


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

ExtraTreesRegressor(n_jobs=-1, random_state=818)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,218.2604,114295.4079,338.0760,0.9997,0.0183,0.0120
1,187.7086,87314.8508,295.4909,0.9998,0.0186,0.0121
2,209.6691,111117.3628,333.3427,0.9997,0.0203,0.0127
3,214.2405,122720.5574,350.3149,0.9997,0.0166,0.0112
4,189.0637,82426.9517,287.1009,0.9998,0.0190,0.0120
5,211.7230,112960.1569,336.0955,0.9997,0.0197,0.0128
6,187.8351,96157.9537,310.0935,0.9998,0.0165,0.0106
7,205.1343,106149.0787,325.8053,0.9998,0.0200,0.0118
8,201.8467,114349.3051,338.1557,0.9997,0.0182,0.0123


...

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Day', 'Month', 'Year',
                                              'Momentum_Indicator',
                                              'High_Low_Change',
                                              'High_Open_Change',
                                              'Low_Open_Change',
                                              'Percent_Change_In_Price',
                                              'Close_High_Ratio',
                                              'Close_Low_Ratio', 'Daily_Range',
                                              'ATR', 'Rolling_Std_Dev', 'RSI',
                                              'MACD', 'Body_Size', 'Upper_Wick',
                                              'Lower_Wick'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(i